In [1]:
import os
import sys
import numpy as np

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake

from pycharmm.lib import charmm as libcharmm

In [2]:
# Ensure that FFT grid is product of small primes 2, 3, 5
def is_factor(n):
    if (n % 2 != 0): return False  # favors even number
    while n:
        flag = False
        for x in (2,3,5):
            if n % x == 0:
               n = n / x
               flag = True
               break

        if flag: continue
        break

    if n == 1: return True
    return False

def checkfft(n, margin = 5):
    n = int(n) + margin
    while 1:
        if is_factor(n): break
        else: n += 1
    return n

In [3]:
def setup_PBC(boxhalf=0.0, protein_segments=[],solvent_resname='TIP3',ions=[],blade=False):
    """input: boxhalf [0.0]
              solute  []
              solvent_resname ['']
              ions []
              blade [False]
    defines the periodic boundary conditions for a cubic volume of boxsize. 
    Uses: crystal_define_cubic(), crystal.build(), image.setup_residue,
    image.setup_segment to construct symmetry operations. 

    If global variable openmm is true
    the image centering is at [boxhalf,boxhalf,boxhalf] otherwise at [0,0,0].
    """
    crystal.define_cubic(boxhalf*2)
    crystal.build(boxhalf)

    if blade: boxhalf = 0.0 # center at origin for blade
    for segment in protein_segments:
        image.setup_segment(boxhalf,boxhalf, boxhalf, segment)
    if len(solvent_resname)>0: image.setup_residue(boxhalf,boxhalf, boxhalf, solvent_resname)
    for ion in ions:
        image.setup_residue(boxhalf, boxhalf, boxhalf, ion)
    # for systems using centering not at origin, translate coordinates by halfbox
    xyz = coor.get_positions()
    xyz += boxhalf
    coor.set_positions(xyz)
    print('Coordinates translated by {} A in each dimension to be consistent with image centering'\
          .format(boxhalf))

    return

In [4]:
# Read in the topology (rtf) and parameter file (prm) for proteins
# equivalent to the CHARMM scripting command: read rtf card name toppar/top_all36_prot.rtf
read.rtf('/home/trowray/acrb/toppar/top_all36_prot.rtf')
# equivalent to the CHARMM scripting command: read param card flexible name toppar/par_all36m_prot.prm
read.prm('/home/trowray/acrb/toppar//par_all36m_prot.prm', flex=True)

# stream in the water/ions parameter using the pycharmm.lingo module
# equivalent to the CHARMM scripting command: stream toppar/toppar_water_ions.str
pycharmm.lingo.charmm_script('stream /home/trowray/acrb/toppar/toppar_water_ions.str')

  
 CHARMM>     read rtf card -
 CHARMM>     name /home/trowray/acrb/toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::/home/trowray/acrb/toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /home/trowray/acrb/toppar//par_all36m_prot.prm -
 CHARMM>     flex
 VOPEN> Attempting to open::/home/trowray/acrb/toppar//par_all36m_prot.prm::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS T

1

In [5]:
pdbid = '/home/trowray/acrb/pdb/1iwg'

# Read the psf and coordinates for the solvated peptide
# Read psf card name pdb/adp+wat.psf
read.psf_card('{}+wat.psf'.format(pdbid))
# read coor pdb name pdb/adp+wat_min.pdb
read.coor_card('{}+wat_min.crd'.format(pdbid),resid=True)

  
 CHARMM>     read psf card -
 CHARMM>     name /home/trowray/acrb/pdb/1iwg+wat.psf
 VOPEN> Attempting to open::/home/trowray/acrb/pdb/1iwg+wat.psf::
 MAINIO> Protein structure file being read from unit  91.
 psf_read_formatted: Reading PSF in the expanded format.
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:     6/27/22     15:27: 8      CREATED BY USER: trowray
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =       13   Number of residues   =    87147
         Number of atoms         =   298743   Number of groups     =    98286
         Number of bonds         =   299166   Number of angles     =   169020
         Number of dihedrals     =   123663   Number of impropers  =     7752
         Number of cross-terms   =     3018   Number of autogens   =    84123
         Number of HB acceptors  =    88398   Number of HB donors  =   173064
         Number of NB 

In [6]:
# Now setup periodic boundaries
# boxsize
stats = coor.stat()
xsize = stats['xmax'] - stats['xmin']
ysize = stats['ymax'] - stats['ymin']
zsize = stats['zmax'] - stats['zmin']
boxsize = max(xsize, ysize, zsize)

# half box size
boxhalf = boxsize / 2.0
# Note we could probably do something to extract the information passed to setup_PPC using pyCHARMM functions
# but I didn't have the time so I just made some generic thing
setup_PBC(boxhalf=boxhalf, protein_segments=['PROA', 'PROB', 'PROC'],
          solvent_resname='TIP3',ions=['CLA', 'SOD', 'POT'],blade=False)

 Crystal Parameters : Crystal Type = CUBI
           A     =  147.42784 B    =  147.42784 C     =  147.42784
           Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
 XBUILD> Building all transformations with a minimum atom-atom
         contact distance of less than   73.71 Angstroms.

 Range of Grid Search for Transformation     1 :
 Lattice Vector A    -2 TO     2
 Lattice Vector B    -2 TO     2
 Lattice Vector C    -2 TO     2


 The number of transformations generated =    26


 Number  Symop   A   B   C   Distance

      1      1  -1  -1  -1     5.2198
      2      1  -1   0  -1     5.9925
      3      1  -1   1  -1    10.6193
      4      1   0  -1  -1     6.6381
      5      1   0   0  -1     3.9916
      6      1   0   1  -1     5.5786
      7      1  -1  -1   0     3.8266
      8      1  -1   0   0     2.9316
      9      1  -1   1   0     7.5235
     10      1   0  -1   0     3.6989
     11      1   0   1   0     3.6989
     12      1  -1  -1   1     7.1270
     1

In [7]:
# Set-up non-bonded parameters
# Now specify nonbonded cutoffs for solvated box
cutnb = min(boxhalf,12)
cutim = cutnb
ctofnb = cutnb - 1.0
ctonnb = cutnb - 3.0
# Determine the appropriate cubic fft grid for this boxsize
fft = checkfft(n=np.ceil(boxhalf)*2,margin=0)
# Set-up the parameters
nb_wPME_vsw = pycharmm.NonBondedScript(cutnb=cutnb, cutim=cutim,
                                       ctonnb=ctonnb, ctofnb=ctofnb,
                                       cdie=True, eps=1,
                                       atom=True, vatom=True,
                                       switch=True, vfswitch=False, vswitch=True,
                                       inbfrq=-1, imgfrq=-1,
                                       ewald=True,pmewald=True,kappa=0.32,
                                       fftx=fft,ffty=fft,fftz=fft,order=4)
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
nb_wPME_vsw.run()
settings.set_warn_level(old_wrnlev)
energy.show()
# Let's also set-up a set of nonbonded parameters using vfswitch instead of vswitch
nb_wPME_vfsw_dict = {'cutnb':cutnb, 
                     'cutim':cutim,
                     'ctonnb':ctonnb, 
                     'ctofnb':ctofnb,
                     'cdie':True,
                     'eps':1,
                     'atom':True, 'vatom':True,
                     'switch':True, 'vfswitch':True, 'vswitch':False,
                     'inbfrq':-1, 'imgfrq':-1,
                     'ewald':True,'pmewald':True,'kappa':0.32,
                     'fftx':fft,'ffty':fft,'fftz':fft,'order':4}
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
pycharmm.NonBondedScript(**nb_wPME_vfsw_dict).run()
settings.set_warn_level(old_wrnlev)
energy.show()
# Now go back to the original nonbonded parameters
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
nb_wPME_vsw.run()
settings.set_warn_level(old_wrnlev)
energy.show()

  
 CHARMM>     nbonds cutnb 12 -
 CHARMM>     cutim 12 -
 CHARMM>     ctonnb 9.0 -
 CHARMM>     ctofnb 11.0 -
 CHARMM>     cdie -
 CHARMM>     eps 1 -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     switch -
 CHARMM>     vswitch -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1 -
 CHARMM>     ewald -
 CHARMM>     pmewald -
 CHARMM>     kappa 0.32 -
 CHARMM>     fftx 150 -
 CHARMM>     ffty 150 -
 CHARMM>     fftz 150 -
 CHARMM>     order 4

 SELECTED IMAGES ATOMS BEING CENTERED ABOUT 73.713920 73.713920 73.713920

 <MKIMAT2>: updating the image atom lists and remapping
 Transformation   Atoms  Groups  Residues  Min-Distance
    1  N1N1N1R1 has      30      10      10        1.21
    2  N1Z0N1R1 has     867     289     289        0.52
    3  N1P1N1R1 has      12       4       4        4.58
    4  Z0N1N1R1 has     777     259     259        1.12
    5  Z0Z0N1R1 has   21253    7085    7085        0.05
    6  Z0P1N1R1 has     828     276     276        0.82
    7  N1N1Z0R1 has     9

In [8]:
def run_md(useomm=False,useblade=False,nequil=1000,nsteps=5000,nsavc=100,leap=True,lang=True):
    if useomm: append='omm'
    elif useblade: append='blade'
    dyn.set_fbetas(np.full((psf.get_natom()),1.0,dtype=float))
   
    res_file = pycharmm.CharmmFile(file_name='/home/trowray/acrb/res/1iwg.res'.format(pdbid), file_unit=2,
                                   formatted=True,read_only=False)
    lam_file = pycharmm.CharmmFile(file_name='/home/trowray/acrb/res/1iwg.lam'.format(pdbid), 
                                   file_unit=3,
                                   formatted=False,read_only=False)
    my_dyn = pycharmm.DynamicsScript(leap=leap, lang=lang, start=True,
                                     nstep=nequil, timest=0.002,
                                     firstt=298.0, finalt=298.0, tbath=298.0,
                                     tstruc=298.0,
                                     teminc=0.0, twindh=0.0, twindl=0.0,
                                     iunwri=res_file.file_unit,
                                     iunlam=lam_file.file_unit,
                                     inbfrq=-1, imgfrq=-1,
                                     iasors=0, iasvel=1, ichecw=0, iscale=0,
                                     iscvel=0,echeck=-1, nsavc=0, nsavv=0, nsavl=0, ntrfrq=0,
                                     isvfrq=nsavc,
                                     iprfrq=2*nsavc, nprint=nsavc, ihtfrq=0, ieqfrq=0,
                                     ilbfrq=0,ihbfrq=0,
                                     omm=useomm, blade=useblade)
    my_dyn.run()

    res_file.close()
    lam_file.close()
    # open unit 2 write form name res/{}.res
    res_file = pycharmm.CharmmFile(file_name='/home/trowray/acrb/res/1iwg.res'.format(pdbid), file_unit=2,
                                   formatted=True,read_only=False)
    lam_file = pycharmm.CharmmFile(file_name='/home/trowray/acrb/res/1iwg.lam'.format(pdbid), 
                                   file_unit=3,
                                   formatted=False,read_only=False)
    # open unit 1 write file name dcd/{}.dcd
    dcd_file = pycharmm.CharmmFile(file_name='/home/trowray/acrb/dcd/1iwg_omm.dcd'.format(pdbid, append), file_unit=1,
                                   formatted=False,read_only=False)

    my_dyn = pycharmm.DynamicsScript(leap=leap, lang=lang, start=False, restart = True,
                                     nstep=nsteps, timest=0.002,
                                     firstt=298.0, finalt=298.0, tbath=298.0,
                                     tstruc=298.0,
                                     teminc=0.0, twindh=0.0, twindl=0.0,
                                     iunwri=res_file.file_unit,
                                     iunrea=res_file.file_unit,
                                     iuncrd=dcd_file.file_unit,
                                     iunlam=lam_file.file_unit,
                                     inbfrq=-1, imgfrq=-1,
                                     iasors=0, iasvel=1, ichecw=0, iscale=0,
                                     iscvel=0,echeck=-1, nsavc=nsavc, nsavv=0, nsavl=0, ntrfrq=0,
                                     isvfrq=nsavc,
                                     iprfrq=2*nsavc, nprint=nsavc, ihtfrq=0, ieqfrq=0,
                                     ilbfrq=0,ihbfrq=0,
                                     omm=useomm, blade=useblade)
    my_dyn.run()

    res_file.close()
    lam_file.close()
    dcd_file.close()
    return

In [9]:
# Set-up short dynamics
if not os.path.isdir('res'): os.system('mkdir res')
if not os.path.isdir('dcd'): os.system('mkdir dcd')
# Check to see if cuda is available to run BLaDE
import torch
cuda  = torch.cuda.is_available()
num_devices = torch.cuda.device_count()
if cuda:
    print('Running CHARMM/BLaDE MD example on computer with {} CUDA devices'.format(num_devices))
    print('Running on device {} which is a {}'.format(torch.cuda.current_device(),torch.cuda.get_device_name(torch.cuda.current_device())))
    run_md(useblade = 'prmc pref 1 iprs 100 prdv 100')
else: print('Example not run, no CUDA devices available')

/export/apps/CentOS7/anaconda/2022/envs/pycharmm-0.5/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Example not run, no CUDA devices available


In [10]:
# Set-up short dynamics
if not os.path.isdir('res'): os.system('mkdir res')
if not os.path.isdir('dcd'): os.system('mkdir dcd')
# Check to see if cuda is available to run OpenMM
import torch
cuda  = torch.cuda.is_available()
num_devices = torch.cuda.device_count()
if cuda:
    print('Running CHARMM/OpenMM MD example on computer with {} CUDA devices'.format(num_devices))
    print('Running on device {} which is a {}'.format(torch.cuda.current_device(),torch.cuda.get_device_name(torch.cuda.current_device())))
else: print('No CUDA devices available, using either CPU or OpenCL')
run_md(useomm = 'gamma 2 prmc pref 1 iprsfrq 100')

No CUDA devices available, using either CPU or OpenCL
 VOPEN> Attempting to open::/home/trowray/acrb/res/1iwg.res::
 VOPEN> Attempting to open::/home/trowray/acrb/res/1iwg.lam::
  
 CHARMM>     dynamics leap -
 CHARMM>     lang -
 CHARMM>     start -
 CHARMM>     nstep 1000 -
 CHARMM>     timest 0.002 -
 CHARMM>     firstt 298.0 -
 CHARMM>     finalt 298.0 -
 CHARMM>     tbath 298.0 -
 CHARMM>     tstruc 298.0 -
 CHARMM>     teminc 0.0 -
 CHARMM>     twindh 0.0 -
 CHARMM>     twindl 0.0 -
 CHARMM>     iunwri 2 -
 CHARMM>     iunlam 3 -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1 -
 CHARMM>     iasors 0 -
 CHARMM>     iasvel 1 -
 CHARMM>     ichecw 0 -
 CHARMM>     iscale 0 -
 CHARMM>     iscvel 0 -
 CHARMM>     echeck -1 -
 CHARMM>     nsavc 0 -
 CHARMM>     nsavv 0 -
 CHARMM>     nsavl 0 -
 CHARMM>     ntrfrq 0 -
 CHARMM>     isvfrq 100 -
 CHARMM>     iprfrq 200 -
 CHARMM>     nprint 100 -
 CHARMM>     ihtfrq 0 -
 CHARMM>     ieqfrq 0 -
 CHARMM>     ilbfrq 0 -
 CHARMM>     ihbfrq 